In [1]:
%load_ext autoreload
%autoreload 2
import eval_models
import pandas as pd
import numpy as np
from boruta import BorutaPy
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
X_dig_train,X_dig_test,y_dig_train,y_dig_test=eval_models.get_train_test("digits")

## No standarization at all

In [3]:
RF = RandomForestClassifier(n_jobs=-1, max_depth=6)
feat_select = BorutaPy(RF, n_estimators='auto', verbose=0)
feat_select.fit(X_dig_train.values,y_dig_train)
X_dig_train_boruta=X_dig_train.iloc[:,feat_select.support_]
X_dig_test_boruta=X_dig_test.iloc[:,feat_select.support_]

In [4]:
noscaling_res=eval_models.get_models_ba(X_dig_train_boruta,
                              X_dig_test_boruta,
                              y_dig_train,
                              y_dig_test
                             )

LogisticRegression 0.9753
RandomForestClassifier 0.9753
AdaBoostClassifier 0.9667
LGBMClassifier 0.9813
XGBClassifier 0.978
1163  features in the dataset


In [5]:
X_train_no_corr=eval_models.delete_corr(X_dig_train_boruta)#,verbose=True)

In [6]:
noscaling_res_cor=eval_models.get_models_ba(X_train_no_corr,
                              X_dig_test_boruta,
                              y_dig_train,
                              y_dig_test
                             )

LogisticRegression 0.9747
RandomForestClassifier 0.9793
AdaBoostClassifier 0.9667
LGBMClassifier 0.9813
XGBClassifier 0.978
1116  features in the dataset


## Standarization after Boruta

In [7]:
X_dig_train_boruta_sc,X_dig_test_boruta_sc=eval_models.standarize(X_dig_train_boruta,X_dig_test_boruta)

In [8]:
X_train_no_corr2=eval_models.delete_corr(X_dig_train_boruta_sc)

In [9]:
noscaling_res_cor=eval_models.get_models_ba(X_train_no_corr2,
                              X_dig_test_boruta_sc,
                              y_dig_train,
                              y_dig_test
                             )

LogisticRegression 0.976
RandomForestClassifier 0.976
AdaBoostClassifier 0.9667
LGBMClassifier 0.9807
XGBClassifier 0.978
1116  features in the dataset


## Standarization before Boruta

In [10]:
X_dig_train_sc,X_dig_test_sc=eval_models.standarize(X_dig_train,X_dig_test)

In [11]:
### (suggested max_depth between 3 and 7) podobno
RF = RandomForestClassifier(n_jobs=-1, max_depth=7)
feat_select = BorutaPy(RF, n_estimators='auto', verbose=0)
feat_select.fit(X_dig_train_sc.values,y_dig_train)
X_dig_train_sc_boruta=X_dig_train_sc.iloc[:,feat_select.support_]
X_dig_test_sc_boruta=X_dig_test_sc.iloc[:,feat_select.support_]

In [12]:
scaled_res=eval_models.get_models_ba(X_dig_train_sc_boruta,
                              X_dig_test_sc_boruta,
                              y_dig_train,
                              y_dig_test
                             )

LogisticRegression 0.974
RandomForestClassifier 0.9767
AdaBoostClassifier 0.9687
LGBMClassifier 0.9787
XGBClassifier 0.9747
1139  features in the dataset


In [13]:
X_dig_train_sc_no_corr=eval_models.delete_corr(X_dig_train_sc_boruta)#,verbose=True)

In [14]:
scaled_res_cor=eval_models.get_models_ba(X_dig_train_sc_no_corr,
                              X_dig_test_sc_boruta,
                              y_dig_train,
                              y_dig_test
                             )

LogisticRegression 0.972
RandomForestClassifier 0.9787
AdaBoostClassifier 0.9687
LGBMClassifier 0.9787
XGBClassifier 0.9747
1092  features in the dataset


## Cross validation (with standarization before Boruta)

In [15]:
kf = KFold(n_splits=5,shuffle=True)
models_res=[]
for train_idx, val_idx in tqdm(kf.split(X_dig_train)):
    X_train=X_dig_train.iloc[train_idx,:]
    X_val=X_dig_train.iloc[val_idx,:]
    y_train=y_dig_train[train_idx]
    y_val=y_dig_train[val_idx]
    
    X_train_sc,X_val_sc=eval_models.standarize(X_train,X_val)
    
    RF = RandomForestClassifier(n_jobs=-1, max_depth=6)
    feat_select = BorutaPy(RF, n_estimators='auto', verbose=0)
    feat_select.fit(X_train.values,y_train)
    #zapisywać np do pliku jakie feature wybrane i zostawić tylko te najczęściej wybierane, na ten moment 3/5 zobaczmy
    
    X_train_sc_boruta=X_train_sc.iloc[:,feat_select.support_]
    X_val_sc_boruta=X_val_sc.iloc[:,feat_select.support_]
    
    with open('../data/boruta/digits-features.csv', 'a') as f:
        np.savetxt(f, np.where(feat_select.support_), delimiter=",", fmt='%i')
    f.close()
    
    X_train_sc_boruta_no_corr=eval_models.delete_corr(X_train_sc_boruta)

    no_of_features=X_train_sc_boruta_no_corr.shape[1]
    tmp=eval_models.get_models_ba(X_train_sc_boruta_no_corr,X_val_sc_boruta,y_train,y_val)
    models_res.append((tmp,no_of_features))

0it [00:00, ?it/s]

LogisticRegression 0.9756
RandomForestClassifier 0.9761
AdaBoostClassifier 0.9735
LGBMClassifier 0.9766


1it [19:30, 1170.51s/it]

XGBClassifier 0.978
1040  features in the dataset
LogisticRegression 0.9767
RandomForestClassifier 0.9657
AdaBoostClassifier 0.9722
LGBMClassifier 0.9744


2it [36:42, 1089.23s/it]

...

XGBClassifier 0.9833
1034  features in the dataset
LogisticRegression 0.9814
RandomForestClassifier 0.9775
AdaBoostClassifier 0.9741
LGBMClassifier 0.9814


4it [1:16:09, 1155.48s/it]

XGBClassifier 0.9772
1056  features in the dataset
LogisticRegression 0.9786
RandomForestClassifier 0.9714
AdaBoostClassifier 0.972
LGBMClassifier 0.9752


5it [1:34:29, 1133.81s/it]

XGBClassifier 0.9775
1042  features in the dataset


### Having run Boruta x times we will select only those variables which have been selected at least 95% of the time
remember to specify x_times! according to the number of tests you've made

In [16]:
x_times=5
boruta_all_results=pd.read_csv('../data/boruta/digits-features.csv',header=None,sep="\n")
boruta_all_results=boruta_all_results.iloc[:,0].str.split(',', expand=True)
boruta_all_array=np.array(list(filter(None, np.array(boruta_all_results).flatten()))).astype(np.int64)
unique, counts=np.unique(np.array(boruta_all_array).flatten(),return_counts=True)
columns_boruta= unique[counts>=x_times*0.95]
X_dig_train_boruta_multiple=X_dig_train.iloc[:,columns_boruta]
X_dig_train_boruta_multiple.head()

,2,12,14,16,25,34,38,48,53,66,...,4963,4966,4967,4970,4976,4977,4979,4980,4981,4991
2868,0,0,991,0,0,0,987,0,332,980,...,0,0,0,0,0,972,987,0,770,0
4630,0,645,991,0,983,0,0,0,0,983,...,991,991,0,0,0,0,662,0,0,0
1870,0,0,987,0,983,801,987,976,976,980,...,0,0,0,0,980,0,0,0,0,0
353,0,877,991,0,0,0,529,0,0,987,...,991,968,0,0,0,991,0,0,937,0
3582,0,0,0,0,831,0,0,0,0,570,...,991,991,0,0,0,874,0,0,0,0


In [17]:
eval_models.get_models_ba(X_dig_train_boruta_multiple,
                              X_dig_test,
                              y_dig_train,
                              y_dig_test,
                          n_estimators=250
                             )

LogisticRegression 0.972
RandomForestClassifier 0.972
AdaBoostClassifier 0.9747
LGBMClassifier 0.9813
XGBClassifier 0.9807
954  features in the dataset


,Classifier,BA score
0,LogisticRegression,0.972000
1,RandomForestClassifier,0.972000
2,AdaBoostClassifier,0.974667
3,LGBMClassifier,0.981333
4,XGBClassifier,0.980667


I was impossible to run VIF on digits dataset :(

In [18]:
X_dig_train_scaled,X_dig_test_scaled=eval_models.standarize(X_dig_train,X_dig_test)
X_dig_train_scaled_boruta_multiple=X_dig_train_scaled.iloc[:,columns_boruta]
X_dig_train_scaled_boruta_multiple_corr=eval_models.delete_corr(X_dig_train_scaled_boruta_multiple)

In [19]:
eval_models.get_models_ba(X_dig_train_scaled_boruta_multiple_corr,
                              X_dig_test_scaled,
                              y_dig_train,
                              y_dig_test,
                          n_estimators=250)

LogisticRegression 0.9747
RandomForestClassifier 0.9753
AdaBoostClassifier 0.9747
LGBMClassifier 0.9787
XGBClassifier 0.9807
912  features in the dataset


,Classifier,BA score
0,LogisticRegression,0.974667
1,RandomForestClassifier,0.975333
2,AdaBoostClassifier,0.974667
3,LGBMClassifier,0.978667
4,XGBClassifier,0.980667
